# UDP 生成自由能计算任务\n\n本笔记本用于计算UDP（尿苷二磷酸）在特定条件下的生成自由能（ΔfG'°）:\n- 温度: 298K\n- pH: 10\n- pMg: 6\n- 离子强度: 0.3M

In [1]:
# 导入必要的库\nimport numpy as np\nimport pandas as pd\nfrom equilibrator_api import ComponentContribution, Q_\nfrom rdkit import Chem\nfrom rdkit.Chem import rdMolDescriptors\n\n# 抑制RDKit警告\nfrom rdkit import RDLogger\nlg = RDLogger.logger()\nlg.setLevel(RDLogger.CRITICAL)\n\nprint("正在初始化ComponentContribution...")\ncc = ComponentContribution()\nprint("ComponentContribution初始化完成")

In [2]:
# 尝试查找UDP化合物\ncompound_name = "UDP"\n\ntry:\n    # 尝试通过不同的方式查找UDP\n    udp_compound = cc.get_compound(compound_name)\n    if udp_compound is None:\n        udp_compound = cc.get_compound(f"kegg:{compound_name}")\n    if udp_compound is None:\n        udp_compound = cc.get_compound(f"chebi:{compound_name}")\n    if udp_compound is None:\n        udp_compound = cc.get_compound_by_inchi(compound_name)\n    \n    if udp_compound is not None:\n        print(f"成功找到UDP化合物: {udp_compound}")\n        print(f"ID: {udp_compound.id}")\n        print(f"InChI: {udp_compound.inchi}")\n        print(f"InChI Key: {udp_compound.inchi_key}")\n        print(f"名称: {udp_compound.name}")\n    else:\n        print("未能找到UDP化合物，请尝试其他名称或格式")\n        # 尝试其他可能的UDP别名\n        possible_names = ["uridine_diphosphate", "uridine_5'-diphosphate", "UMP", "UTP", "ADP", "GDP"]\n        for name in possible_names:\n            compound = cc.get_compound(name)\n            if compound is not None:\n                print(f"找到类似化合物 {name}: {compound}")\nexcept Exception as e:\n    print(f"查找UDP化合物时出错: {e}")

In [3]:
# 如果上面没找到UDP，尝试更具体的搜索\nudp_compound = None\n\n# 尝试使用KEGG ID\ntry:\n    udp_compound = cc.get_compound("kegg:C00015")  # UDP的KEGG ID\n    if udp_compound is not None:\n        print(f"通过KEGG ID找到UDP: {udp_compound.name}")\nexcept:\\n    pass\n\n# 如果还是没找到，尝试其他方式\nif udp_compound is None:\n    try:\n        udp_compound = cc.get_compound("uridine diphosphate")\n        if udp_compound is not None:\n            print(f"通过全名找到UDP: {udp_compound.name}")\n    except:\\n        pass\n\nif udp_compound is None:\n    print("仍然无法找到UDP化合物，尝试使用更广泛的搜索")\nelse:\n    print(f"成功获取UDP化合物信息: {udp_compound}")

In [4]:
# 计算UDP在指定条件下的生成自由能\nif udp_compound is not None:\n    # 设定条件\n    temp = 298  # K\n    ph = 10\n    pmg = 6\n    ionic_strength = 0.3  # M\n    \n    print(f"开始计算UDP在以下条件下的生成自由能:")\n    print(f"温度: {temp} K")\n    print(f"pH: {ph}")\n    print(f"pMg: {pmg}")\n    print(f"离子强度: {ionic_strength} M")\n    \n    try:\n        # 使用transform方法将标准条件转换到指定条件\n        transformed_compound = udp_compound.transform(p_h=Q_(ph), p_mg=Q_(pmg), ionic_strength=Q_(f'{ionic_strength}M'), temperature=Q_(f'{temp}K'))\n        \n        # 获取标准生成自由能\n        standard_dgf = udp_compound.standard_dgf\n        \n        # 计算在指定条件下的生成自由能\n        adjusted_dgf = standard_dgf + transformed_compound\n        \n        print(f"\nUDP在指定条件下的生成自由能 (ΔfG'°): {adjusted_dgf.value} kJ/mol")\n        print(f"标准生成自由能 (ΔfG°): {standard_dgf.value} kJ/mol")\n        print(f"校正项: {transformed_compound.value} kJ/mol")\n        \n        # 获取不确定性\n        try:\n            uncertainty = udp_compound.uncertainty\n            print(f"不确定性: ±{uncertainty} kJ/mol")\n        except:\\n            print("未找到不确定性数据")\n            \n    except Exception as e:\n        print(f"计算过程中出现错误: {e}")\n        \n        # 尝试另一种方法\n        try:\n            print("\n尝试使用另一种方法计算...")\n            # 使用compound的transformed formation energy方法\n            formation_energy = udp_compound.transformed_formation_energy(p_h=Q_(ph), p_mg=Q_(pmg), ionic_strength=Q_(f'{ionic_strength}M'), temperature=Q_(f'{temp}K'))\n            print(f"UDP在指定条件下的生成自由能 (ΔfG'°): {formation_energy.value} kJ/mol")\n        except Exception as e2:\n            print(f"第二种方法也失败了: {e2}")\nelse:\n    print("由于找不到UDP化合物，无法进行计算")